In [6]:
import great_expectations as ge
from csv import reader,writer
import os
import datetime

In [7]:
def read_csv_file(file_path : str):
    return ge.read_csv(file_path)

In [8]:
def validate_expectations(data_frame,expectation_file_path):
    return data_frame.validate(expectation_suite= expectation_file_path)

In [9]:
HEADER=["FILE_NAME","COLUMN_NAME","TEST_TYPE","TEST_STATUS","TOTAL_EVALUATIONS","EVALUATIONS_PASSED","EVALUATIONS_FAILED","SUCCESS_RATIO","VALIDATION_TIME","CURRENT_TIME"]

In [10]:
# skip first line i.e. read header first and then iterate over each row od csv as a list
with open('./config/file_config.csv', 'r') as read_obj:
    csv_file = open("./output/validation_result.csv", "w")
    csv_file.write(",".join(HEADER))
    csv_file.write("\n")
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    # Check file as empty
    if header != None:
        # Iterate over each row after the header in the csv
        for file_path,expectation_file_path in csv_reader:
            
            # row variable is a list that represents a row in csv
            file_name=os.path.basename(file_path)
            current_time = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
            df = read_csv_file(file_path)
            result = validate_expectations(df,expectation_file_path)
            total_evaluations=str(result["statistics"]["evaluated_expectations"])
            evaluations_passed=str(result["statistics"]["successful_expectations"])
            evaluations_failed=str(result["statistics"]["unsuccessful_expectations"])
            success_ratio=str(result["statistics"]["success_percent"])
            validation_time=result["meta"]["validation_time"]
            for test in result["results"]:
                test_type=test["expectation_config"]["expectation_type"]
                test_status=str(test["success"])
                column_name=test["expectation_config"]["kwargs"]["column"]
                line = [file_name,column_name,test_type,test_status,total_evaluations,evaluations_passed,evaluations_failed,success_ratio,validation_time,current_time]
                csv_file.write(','.join(line))
                csv_file.write('\n')
                
    csv_file.close()